<a href="https://colab.research.google.com/github/heidrunmuehle/KalenderApp/blob/main/CalenderAppBasicV1_Read_Talk_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""CalendarAppBasicV1/Read/Talk/Gemini.py

Fixed version of the calendar authentication script
"""

# Install required packages
!apt-get update
!apt-get install -y libportaudio2 libportaudiocpp0 portaudio19-dev
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib google-auth google-generativeai SpeechRecognition gTTS playsound pyaudio


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,632 B in 3s (1,082 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... D

In [2]:

import google.generativeai as genai
from google_auth_oauthlib.flow import Flow
from googleapiclient.discovery import build
import datetime
import os
import speech_recognition as sr
from gtts import gTTS
import playsound
from IPython.display import Audio, display

In [7]:
client_secret_file = 'client_secret_2_710076331232-dhd20rctbckgahvje160tmn019im73g6.apps.googleusercontent.com.json'


In [8]:
# Load the client configuration from the JSON file
with open(client_secret_file, 'r') as file:
    client_config = json.load(file)

In [9]:


# ------------------- Google Calendar API Configuration -------------------
# This scope allows readonly access to Google Calendar
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

In [10]:


# ------------------- OAuth Flow Setup -------------------
flow = Flow.from_client_config(
    client_config=client_config,
    scopes=SCOPES,
    redirect_uri=client_config['web']['redirect_uris'][0]  # Use the first redirect URI from the file
)

In [11]:


# ------------------- Generate Authorization URL -------------------
auth_url, _ = flow.authorization_url(
    prompt='consent',       # Force consent prompt
    access_type='offline'   # Enable refresh token
)

print(">>> PLEASE FOLLOW THIS LINK TO AUTHORIZE ACCESS <<<")
print(auth_url)
print("\n>>> AFTER AUTHORIZATION, GOOGLE REDIRECTS TO http://localhost:8088 <<<")
print(">>> THIS REDIRECT WILL FAIL IN COLAB, BUT THE ADDRESS BAR WILL HAVE 'code=' <<<")
print(">>> COPY THE VALUE AFTER 'code=' AND PASTE IT BELOW <<<")


>>> PLEASE FOLLOW THIS LINK TO AUTHORIZE ACCESS <<<
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=710076331232-dhd20rctbckgahvje160tmn019im73g6.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8088&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=BejoBlBM2HaMhBtOJKcqHf7RKkuC2Q&prompt=consent&access_type=offline

>>> AFTER AUTHORIZATION, GOOGLE REDIRECTS TO http://localhost:8088 <<<
>>> THIS REDIRECT WILL FAIL IN COLAB, BUT THE ADDRESS BAR WILL HAVE 'code=' <<<
>>> COPY THE VALUE AFTER 'code=' AND PASTE IT BELOW <<<


In [12]:

# ------------------- Exchange authorization code for credentials -------------------
auth_code = input("\nPlease paste the value from 'code=' here: ")
flow.fetch_token(code=auth_code)
creds = flow.credentials


Please paste the value from 'code=' here: 4/0AQSTgQFafVB_u8DK-xkSiTvZoR2tX9oUAaXUHtQIIYuRSf4RalBAFCYN-ravzaWp_e7kzg&scope=https://www.googleapis.com/auth/calendar.readonly


In [14]:


# ------------------- Calendar Service Setup -------------------
service = build('calendar', 'v3', credentials=creds)

# ------------------- Helper Functions -------------------
def get_upcoming_events(num_events=10):
    """Fetch upcoming events from the user's primary calendar."""
    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
    print(f'Getting the upcoming {num_events} events')

    events_result = service.events().list(
        calendarId='primary',
        timeMin=now,
        maxResults=num_events,
        singleEvents=True,
        orderBy='startTime'
    ).execute()

    events = events_result.get('items', [])

    if not events:
        return "No upcoming events found."

    event_list = []
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        event_list.append(f"{start}: {event['summary']}")

    return "\n".join(event_list)

def speak_text(text):
    """Convert text to speech and play it."""
    tts = gTTS(text=text, lang='en')
    filename = "output.mp3"
    tts.save(filename)

    # Display audio for playback in Colab
    display(Audio(filename, autoplay=True))

    # Clean up the file
    os.remove(filename)

def listen_for_command():
    """Listen for voice commands using the microphone."""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening for command...")
        audio = r.listen(source)

    try:
        command = r.recognize_google(audio)
        print(f"Recognized: {command}")
        return command.lower()
    except sr.UnknownValueError:
        print("Could not understand audio")
        return ""
    except sr.RequestError as e:
        print(f"Could not request results; {e}")
        return ""

# ------------------- Main Application Loop -------------------
def run_calendar_assistant():
    """Run the main calendar assistant loop."""
    speak_text("Calendar Assistant is ready. You can ask about your upcoming events.")

    while True:
        command = listen_for_command()

        if "events" in command or "calendar" in command or "schedule" in command:
            events = get_upcoming_events(5)  # Get next 5 events
            speak_text("Here are your upcoming events: " + events)

        elif "exit" in command or "quit" in command or "goodbye" in command:
            speak_text("Goodbye!")
            break

        elif command:
            speak_text("I didn't understand that. You can ask about your upcoming events or say goodbye to exit.")

# ------------------- Test Calendar Access -------------------
# Test by fetching upcoming events
try:
    events = get_upcoming_events(3)  # Get next 3 events
    print("Your upcoming events:")
    print(events)

    # Uncomment to enable voice interaction
    # run_calendar_assistant()

except Exception as e:
    print(f"An error occurred: {e}")


Getting the upcoming 3 events
Your upcoming events:
2025-03-04T09:30:00+01:00:  StandUp 506 Development Team 
2025-03-04T10:00:00+01:00: Kick Off Meeting Testing Mandanten Level 
2025-03-04T10:30:00+01:00: Refinment 


JSON file not found, using hardcoded configuration
